In [4]:
import pandas as pd
import numpy as np
import nibabel as nib
from nilearn import input_data, datasets
import networkx as nx
from sklearn.model_selection import train_test_split
from torch_geometric.data import Data, DataLoader
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, global_mean_pool
from fastdtw import fastdtw
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, classification_report
from torch.nn import BatchNorm1d, Dropout
import os
from nilearn import plotting
from nilearn import image
import matplotlib.pyplot as plt
import warnings
import pprint
# Ignore all warnings (not recommended unless you know what you are doing)
warnings.filterwarnings("ignore")
from tqdm import tqdm

/Users/vinoth/opt/anaconda3/lib/python3.9/site-packages/nilearn/input_data/__init__.py:23: FutureWarning: The import path 'nilearn.input_data' is deprecated in version 0.9. Importing from 'nilearn.input_data' will be possible at least until release 0.13.0. Please import from 'nilearn.maskers' instead.
  warnings.warn(message, FutureWarning)


In [15]:
from nilearn import image
import pandas as pd
import os
from sklearn.model_selection import train_test_split
from tqdm import tqdm

# Load the CSV file
csv_file = pd.read_csv(r"/Users/vinoth/PycharmProjects/paper_implementation/Dataset/source/mri_images/ABIDE_pcp/Phenotypic_V1_0b_preprocessed1.csv")
csv_file['DX_GROUP'].replace({1: 0, 2: 1}, inplace=True)
train_df, test_df = train_test_split(csv_file, test_size=0.1, random_state=42)

# Data Preprocessing
for idx, row in tqdm(enumerate(train_df.itertuples()), total=len(train_df)):
    try:
        # Load the MRI image
        mri_dir = r"/Users/vinoth/PycharmProjects/paper_implementation/Dataset/source/mri_images/ABIDE_pcp/cpac/nofilt_noglobal/"
        image_path = os.path.join(mri_dir, row.FILE_ID + "_func_preproc.nii.gz")
        mri_img = image.load_img(image_path)

        # Get the count of time points
        num_time_points = mri_img.shape[3]

        # Segmentation using Multi-Region Growing algorithm
        from nilearn.regions import connected_regions
        regions_img, index = connected_regions(mri_img, min_region_size=500)

        # Count the number of regions
        number_of_regions = len(index)

        # Print MRI image's time point count and region count
        print(f"Image has {num_time_points} time points and {number_of_regions} regions.")

        # Plot the segmented regions (if needed)
        # plotting.plot_roi(regions_img, bg_img=mri_img, title="Segmented Regions")
        # plotting.show()
        
    except Exception as e:
        pass


  0%|          | 1/1000 [00:23<6:33:46, 23.65s/it]

Image has 236 time points and 271 regions.


  0%|          | 4/1000 [00:37<2:14:23,  8.10s/it]

Image has 146 time points and 174 regions.


  0%|          | 5/1000 [01:10<4:11:21, 15.16s/it]

...

Image has 176 time points and 219 regions.


  3%|▎         | 32/1000 [08:07<4:03:23, 15.09s/it]

Image has 176 time points and 227 regions.


  3%|▎         | 32/1000 [08:24<4:14:08, 15.75s/it]


KeyboardInterrupt: 